In [3]:
import torch
import gpytorch
import botorch
import matplotlib.pyplot as plt
import numpy as np

plt.style.use("bmh")

from tqdm.notebook import tqdm

class Hartmann:
    """Generic Hartmann in 3,4,6 dims on [0,1]^d."""
    _params = {
        3: {
            "alpha": torch.tensor([1.0,1.2,3.0,3.2]),
            "A":     torch.tensor([[3.0,10,30],
                                   [0.1,10,35],
                                   [3.0,10,30],
                                   [0.1,10,35]]),
            "P":     torch.tensor([[3689,1170,2673],
                                   [4699,4387,7470],
                                   [1091,8732,5547],
                                   [381,5743,8828]]) * 1e-4
        },
        4: {
            "alpha": torch.tensor([1.0,1.2,3.0,3.2]),
            "A":     torch.tensor([[10,3,17,3.5],
                                   [0.05,10,17,0.1],
                                   [3,3.5,1.7,10],
                                   [17,8,0.05,10]]),
            "P":     torch.tensor([[1312,1696,5569,124],
                                   [2329,4135,8307,3736],
                                   [2348,1451,3522,2883],
                                   [4047,8828,8732,5743]]) * 1e-4
        },
        6: {
            "alpha": torch.tensor([1.0,1.2,3.0,3.2]),
            "A":     torch.tensor([[10,3,17,3.5,1.7,8],
                                   [0.05,10,17,0.1,8,14],
                                   [3,3.5,1.7,10,17,8],
                                   [17,8,0.05,10,0.1,14]]),
            "P":     torch.tensor([[1312,1696,5569,124,8283,5886],
                                   [2329,4135,8307,3736,1004,9991],
                                   [2348,1451,3522,2883,3047,6650],
                                   [4047,8828,8732,5743,1091,381]]) * 1e-4
        }
    }

    def __init__(self, dim=3):
        assert dim in (3,4,6)
        p = self._params[dim]
        self.alpha, self.A, self.P = p["alpha"], p["A"], p["P"]
        self.bounds = torch.stack([torch.zeros(dim), torch.ones(dim)])

    def __call__(self, x: torch.Tensor) -> torch.Tensor:
        # x: (..., d)
        d = x.size(-1)
        inner = ((x.unsqueeze(-2) - self.P)**2 * self.A).sum(dim=-1)  # (..., 4)
        return -(self.alpha * torch.exp(-inner)).sum(dim=-1)

def visualize_gp_belief_and_policy(model, likelihood, policy=None, next_x=None):
    # Disabled for 6D Hartmann function; kept for compatibility
    pass

class GPModel(gpytorch.models.ExactGP, botorch.models.gpytorch.GPyTorchModel):
    _num_outputs = 1

    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

def fit_gp_model(train_x, train_y, num_train_iters=500):
    # Declare the GP
    noise = 1e-4

    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = GPModel(train_x, train_y, likelihood)
    model.likelihood.noise = noise

    # Train the hyperparameters
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    model.train()
    likelihood.train()

    for i in tqdm(range(num_train_iters), desc="Training GP"):
        optimizer.zero_grad()

        output = model(train_x)
        loss = -mll(output, train_y)

        loss.backward()
        optimizer.step()

    model.eval()
    likelihood.eval()

    return model, likelihood

# Initialize Hartmann function (6D) and bounds
hartmann = Hartmann(dim=6)
bounds = hartmann.bounds  # [[0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1]]

# Parameters
n_initial_points = 10
num_queries = 10
num_runs = 10

# Store minimum f(x) for each run
all_min_f_values = []

for run in range(num_runs):
    print(f"\nRun {run + 1}/{num_runs}")
    
    # Set different random seed for each run
    torch.manual_seed(2 + run)
    
    # Initialize with 10 random points in [0, 1]^6
    train_x = torch.rand(size=(n_initial_points, 6))  # Uniformly sampled in [0, 1]^6
    train_y = hartmann(train_x)

    # Print initial points and their function values
    print("Initial points (x1, x2, x3, x4, x5, x6, f(x)):")
    print(torch.hstack([train_x, train_y.unsqueeze(1)]))

    # Fit initial GP model
    model, likelihood = fit_gp_model(train_x, train_y)

    # Track minimum f(x) for this run
    min_f_values = [train_y.min().item()]

    # Run Bayesian optimization
    for i in range(num_queries):
        print(f"Iteration {i}, incumbent x=({train_x[train_y.argmin()].numpy().round(4).tolist()}), f(x)={train_y.min().item():.4f}")

        model, likelihood = fit_gp_model(train_x, train_y)

        policy = botorch.acquisition.analytic.ProbabilityOfImprovement(
            model, best_f=train_y.min(), maximize=False
        )
        
        next_x, acq_val = botorch.optim.optimize_acqf(
            policy,
            bounds=bounds,
            q=1,
            num_restarts=20,
            raw_samples=50,
        )

        next_y = hartmann(next_x)

        # Ensure next_y is 1D with shape (1,)
        next_y = next_y.unsqueeze(0) if next_y.dim() == 0 else next_y

        train_x = torch.cat([train_x, next_x])
        train_y = torch.cat([train_y, next_y])
        
        # Store the minimum f(x)
        min_f_values.append(train_y.min().item())

    # Store results for this run
    all_min_f_values.append(min_f_values)

    # Plot minimum f(x) vs iteration for this run
    plt.figure(figsize=(8, 6))
    plt.plot(range(num_queries + 1), min_f_values, marker='o', linestyle='-', color='b')
    plt.xlabel('Iteration')
    plt.ylabel('Minimum f(x)')
    plt.title(f'Minimum f(x) vs Iteration (Run {run + 1})')
    plt.grid(True)
    plt.savefig(f'min_f_vs_iteration_run_{run + 1}.png')
    plt.close()

# Convert to numpy array for easier computation
all_min_f_values = np.array(all_min_f_values)  # Shape: (num_runs, num_queries + 1)

# Compute mean, std, and 95% CI
mean_f = np.mean(all_min_f_values, axis=0)
std_f = np.std(all_min_f_values, axis=0)
ci_95 = 1.96 * std_f / np.sqrt(num_runs)  # 95% CI: z=1.96 for normal distribution

# Plot convergence with mean, std, and CI
plt.figure(figsize=(10, 6))
iterations = range(num_queries + 1)

# Plot mean
plt.plot(iterations, mean_f, marker='o', linestyle='-', color='b', label='Mean Minimum f(x)')

# Plot 95% CI
plt.fill_between(
    iterations,
    mean_f - ci_95,
    mean_f + ci_95,
    color='b',
    alpha=0.2,
    label='95% Confidence Interval'
)

# Plot mean ± std
plt.fill_between(
    iterations,
    mean_f - std_f,
    mean_f + std_f,
    color='g',
    alpha=0.1,
    label='Mean ± Std'
)

plt.xlabel('Iteration')
plt.ylabel('Minimum f(x)')
plt.title('Convergence Plot: Mean, Std, and 95% CI over 10 Runs (Hartmann 6D)')
plt.grid(True)
plt.legend()
plt.savefig('convergence_plot.png')
plt.close()


Run 1/10
Initial points (x1, x2, x3, x4, x5, x6, f(x)):
tensor([[ 0.6147,  0.3810,  0.6371,  0.4745,  0.7136,  0.6190, -0.0970],
        [ 0.4425,  0.0958,  0.6142,  0.0573,  0.5657,  0.5332, -0.6787],
        [ 0.3901,  0.9088,  0.5334,  0.7073,  0.7116,  0.2050, -1.7305],
        [ 0.3078,  0.9809,  0.0103,  0.4660,  0.4604,  0.8547, -0.0764],
        [ 0.4525,  0.6317,  0.4760,  0.2200,  0.2166,  0.2571, -0.5447],
        [ 0.0458,  0.1755,  0.6177,  0.8291,  0.5246,  0.2708, -0.0501],
        [ 0.7197,  0.3081,  0.3892,  0.2259,  0.3430,  0.0367, -0.0669],
        [ 0.7133,  0.6944,  0.5993,  0.7455,  0.7119,  0.5221, -0.0183],
        [ 0.5530,  0.5382,  0.7668,  0.8359,  0.8591,  0.7898, -0.0090],
        [ 0.3781,  0.4777,  0.3984,  0.7909,  0.5555,  0.9628, -0.0438]])


Training GP:   0%|          | 0/500 [00:00<?, ?it/s]

Iteration 0, incumbent x=([0.39010000228881836, 0.9088000059127808, 0.5333999991416931, 0.7073000073432922, 0.7116000056266785, 0.20499999821186066]), f(x)=-1.7305


Training GP:   0%|          | 0/500 [00:00<?, ?it/s]

Iteration 1, incumbent x=([0.3862999975681305, 0.9114999771118164, 0.5325000286102295, 0.7067999839782715, 0.7114999890327454, 0.2011999934911728]), f(x)=-1.7578


...

Training GP:   0%|          | 0/500 [00:00<?, ?it/s]

Iteration 9, incumbent x=([0.35510000586509705, 0.51419997215271, 0.7386999726295471, 0.45260000228881836, 0.14309999346733093, 0.7508000135421753]), f(x)=-1.0738


/opt/anaconda3/envs/meng/lib/python3.10/site-packages/botorch/optim/optimize.py:677: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.')]
Trying again with a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)


Training GP:   0%|          | 0/500 [00:00<?, ?it/s]

/opt/anaconda3/envs/meng/lib/python3.10/site-packages/botorch/optim/optimize.py:677: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.')]
Trying again with a new set of initial conditions.
  return _optimize_acqf_batch(opt_inputs=opt_inputs)
